In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import matplotlib.pyplot as plt # for plotting
import seaborn           as sb  # for plotting
import pandas            as pd  # for plotting adjacency matrices
import networkx          as nx  # for plotting and pagerank
import collections

from multiprocessing import Pool

colours = ["windows blue", "amber", "pale red", "greyish", "faded green", "dusty purple", "orange", "turquoise", "magenta"]

sb.set()
sb.set_style("white")
sb.set_palette(sb.xkcd_palette(colours))

In [ ]:
# import our simulator
from network_simulation import *

In [ ]:
attacks_per_zone_per_day    = round(1000000000 / 40)
attacks_per_zone_per_hour   = round(attacks_per_zone_per_day / 24)
attacks_per_zone_per_minute = round(attacks_per_zone_per_hour / 60)

the_seed = 21453

# generate a network
G, extra, attacks, hubs, attacker = \
    generate_graph( num_nodes              =    40
                  , num_hubs               =     3
                  , ticks                  = 10 * attacks_per_zone_per_minute
                  , seed                   = the_seed
                  , attacker_at_hub        = False
                  , attacker_activity      =     0.0002 # should we include attacks when we learn?
                  , hub_to_hub_probability =     0.35
                  , hub_fixation           =     0.0
                  )

We split attacks in half and attack one half in the training data, the other one we will try to detect

In [ ]:
# add half of the attacks to the graph
for (source, target) in attacks[:len(attacks)//2]:
    w = 1
    if (source, target) in G.edges:
        w += G.edges[(source, target)]["weight"]
    G.add_edge(source, target, weight = w)

# keep the other half for detection later
kept_attacks = attacks[len(attacks)//2:]

In [ ]:
# out of curiosity, plot the adjacency matrix
plt.matshow(nx.adjacency_matrix(G).todense())

In [ ]:
print(f"Kept {len(kept_attacks)} attacks")
kept_attacks

In [ ]:
print("hubs    ", list(hubs))
print("attacker", attacker)

In [ ]:
# plot the network and mark regular nodes, hubs, and the attacker
colours = { "regular"  : sb.color_palette()[0] # windows blue
          , "hub"      : sb.color_palette()[1] # amber
          , "attacker" : sb.color_palette()[2] # pale red
          }

pos          = { node : (G.nodes[node]["x"], G.nodes[node]["y"]) for node in G.nodes }
node_colours = []
for node in G.nodes:
    if node in hubs:
        node_colours.append(colours["hub"])
    elif node == attacker:
        node_colours.append(colours["attacker"])
    else:
        node_colours.append(colours["regular"])
edges        = G.edges
weights      = [1+log(G[u][v]["weight"]) for u,v in edges]

plt.figure(figsize=(8,8))
nx.draw_networkx(G, pos = pos, node_color = node_colours, width = weights)
plt.tight_layout()
plt.savefig(f"network_{the_seed}.pdf")

In [ ]:
# plot the degrees per node
xys = dict(G.degree)
plt.scatter(xys.keys(), xys.values(), marker = "x")
plt.scatter(hubs, [xys[hub] for hub in hubs], marker = "x", color = sb.color_palette()[1])
plt.scatter([attacker], [xys[attacker]], marker = "x", color = sb.color_palette()[2])
plt.hlines(average(list(xys.values())), xmin=0, xmax=len(xys))
plt.xlim(0, len(xys))

In [ ]:
# find the flow distribution with standard PageRank
pagerank = nx.pagerank(G, alpha = 0.85)

In [ ]:
# plot the pagerank per node
plt.hlines(average(list(pagerank.values())), xmin=0, xmax=len(pagerank))
plt.scatter(pagerank.keys(), pagerank.values(), marker = "x", label = "regular node")
plt.scatter(hubs, [pagerank[hub] for hub in hubs], marker = "x", color = sb.color_palette()[1], label = "hub")
plt.scatter([attacker], [pagerank[attacker]], marker = "x", color = sb.color_palette()[2], label = "attacker")
plt.xlim(0, len(xys))
plt.xlabel("node")
plt.ylabel("pagerank")
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.savefig(f"pagerank_{the_seed}.pdf")

In [ ]:
pagerank[attacker]

In [ ]:
# calculate the transition probabilities
transition_probabilities = dict()

for s in G.nodes:
    transition_probabilities[s] = collections.defaultdict(float)
    w_total = 0
    for t in G.neighbors(s):
        w_total += G.edges[(s,t)]["weight"]
    for t in G.neighbors(s):
        transition_probabilities[s][t] = G.edges[(s,t)]["weight"] / w_total

In [ ]:
def do_random_walk(transition_probabilities, source, target):
    steps = 0
    while source != target and steps < 10000:
        neigbours = list(transition_probabilities[source].items())
        source = np.random.choice( a    = [n for (n,_) in neigbours]
                                 , size = 1
                                 , p    = [p for (_,p) in neigbours]
                                 )[0]
        steps += 1
    return steps

We need to check if there is any node without incoming links, then the random walk would never converge

In [ ]:
no_incoming = set(G.nodes)

for source in transition_probabilities:
    for target in transition_probabilities:
        if target in no_incoming:
            no_incoming.remove(target)

print(f"Nodes without incoming links: {sorted(no_incoming)}")

Calculate all median number of steps between pairs of nodes

In [ ]:
def getMedian(sourceTarget):
    source, target = sourceTarget
    return ( source
           , target
           , np.median([do_random_walk(transition_probabilities, source, target) for _ in range(100)])
           )

In [ ]:
median_steps_connected    = dict()
median_steps_disconnected = dict()

# median number of steps for connected nodes
pairs = list(G.edges)
with Pool() as p:
    medians = p.map(getMedian, pairs)

for source, target, steps in medians:
    median_steps_connected[(source, target)] = steps

# median number of steps for disconnected nodes (new valid connections + attacks)
pairs = set(kept_attacks + extra).difference(set(G.edges))
with Pool() as p:
    attack_medians = p.map(getMedian, pairs)

for source, target, steps in attack_medians:
    median_steps_disconnected[(source, target)] = steps

Let's see if we can detect attacks

In [ ]:
l = list(median_steps_connected.values())
threshold = sorted(l)[:round(0.95*len(l))][-1]
print(threshold)

Check the extra edges

In [ ]:
tn = 0
fp = 0

for (source, target) in extra:
    if (source, target) in G.edges:
        steps = median_steps_connected[(source, target)]
    else:
        steps = median_steps_disconnected[(source, target)]
    
    if steps > threshold:
        fp += 1
    else:
        tn += 1

print(f"Detected {fp}/{len(extra)} as attacks.")

In [ ]:
tp = 0
fn = 0

median_steps_attack = list()

for (target, attacker) in kept_attacks:
    p = transition_probabilities[target][attacker]
    
    if (target, attacker) in G.edges:
        steps = median_steps_connected[(target, attacker)]
    else:
        steps = median_steps_disconnected[(target, attacker)]
    median_steps_attack.append(steps)

    if steps > threshold:
        tp += 1
    else:
        fn += 1

print(f"Detected {tp}/{len(kept_attacks)} attacks.")

Check the F1 score

In [ ]:
print(f"TP: {tp}")
print(f"TN: {tn}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"Precision: {tp / (tp + fp)}")
print(f"Recall: {tp / (tp + fn)}")
print(f"F1 score: {tp / (tp + 0.5 * (fp + fn))}")

In [ ]:
plt.figure(figsize=(16,4))
sb.histplot(list(median_steps_connected.values()), color = sb.color_palette()[0])
sb.histplot(list(median_steps_attack), color = sb.color_palette()[2])
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()
plt.vlines(threshold, ymin, ymax, color = sb.color_palette()[0])
plt.xlim(0,xmax)
plt.ylim(ymin, ymax)